In [48]:
from libpgm.nodedata import NodeData
from libpgm.graphskeleton import GraphSkeleton
from libpgm.discretebayesiannetwork import DiscreteBayesianNetwork
from libpgm.lgbayesiannetwork import LGBayesianNetwork
from libpgm.hybayesiannetwork import HyBayesianNetwork
from libpgm.dyndiscbayesiannetwork import DynDiscBayesianNetwork
from libpgm.tablecpdfactorization import TableCPDFactorization
from libpgm.sampleaggregator import SampleAggregator
from libpgm.pgmlearner import PGMLearner

import numpy as np
#create table
from prettytable import PrettyTable

#read csv
import pandas as pd

In [127]:
data=pd.read_csv("./csv/all_reviews.csv")
sentimental=pd.read_csv("./csv/text_views.csv")
data_sentimental=pd.read_csv("./csv/all_reviews.csv")

del data_sentimental['target']
data_sentimental['target']=sentimental['sentimental']

data.query('overall > 0',inplace=True)
data.query('clean > 0',inplace=True)
data.query('value >0',inplace=True)
data.query('room >0',inplace=True)
data.query('location >0',inplace=True)
data.query('check >0',inplace=True)
data.query('service >0',inplace=True)
data.query('business >0',inplace=True)
#data['id'] = np.arange(len(data))
del data['id']
data['id'] = np.arange(len(data))

cols=data.columns.tolist()
cols= cols[-1:]+cols[:-1]
data=data[cols]
data.to_csv("./csv/ult_views.csv",index=False)
all_data=len(data)
data.head(5)

,id,hotel,reader,helpful,overall,value,room,location,clean,check,service,business,target
0,0,hotel_231505,-1,-1,5,5,5,5,5,5,5,5,good
1,1,hotel_231505,-1,-1,1,1,1,3,2,1,3,3,bad
2,2,hotel_231505,-1,-1,3,3,2,3,4,2,2,3,bad
5,3,hotel_231505,4,4,4,4,4,4,4,4,3,5,good
7,4,hotel_231505,3,3,4,4,4,3,4,5,4,3,good


In [128]:
data_sentimental.query('overall > 0',inplace=True)
data_sentimental.query('clean > 0',inplace=True)
data_sentimental.query('value >0',inplace=True)
data_sentimental.query('room >0',inplace=True)
data_sentimental.query('location >0',inplace=True)
data_sentimental.query('check >0',inplace=True)
data_sentimental.query('service >0',inplace=True)
data_sentimental.query('business >0',inplace=True)
#data['id'] = np.arange(len(data))
del data_sentimental['id']
data_sentimental['id'] = np.arange(len(data_sentimental))

cols=data_sentimental.columns.tolist()
cols= cols[-1:]+cols[:-1]
data_sentimental=data_sentimental[cols]
data_sentimental.to_csv("./csv/ult_views_sentimental.csv",index=False)
all_data_sent=len(data_sentimental)
data_sentimental.head(5)

,id,hotel,reader,helpful,overall,value,room,location,clean,check,service,business,target
0,0,hotel_231505,-1,-1,5,5,5,5,5,5,5,5,good
1,1,hotel_231505,-1,-1,1,1,1,3,2,1,3,3,bad
2,2,hotel_231505,-1,-1,3,3,2,3,4,2,2,3,bad
5,3,hotel_231505,4,4,4,4,4,4,4,4,3,5,good
7,4,hotel_231505,3,3,4,4,4,3,4,5,4,3,good


In [129]:
#istruzione da commentare se si vuole ottenere il training per data con l'assunzione
data=data_sentimental
#bad=data.query('target == " bad" and overall >0')
#good=data.query('target == " good"')

bad=data.query('target == "bad" and overall >0')
good=data.query('target == "good"')
bad_n=len(bad)
good_n=len(good)

print(float(good_n)/all_data,float(bad_n)/all_data)



(0.6191283965086513, 0.38087160349134874)


In [130]:
#numero di volte per ogni valore di clean dato good
clean_g_1=data.query('target == "good" and clean == 1')
clean_g_2=data.query('target == "good" and clean == 2')
clean_g_3=data.query('target == "good" and clean == 3')
clean_g_4=data.query('target == "good" and clean == 4')
clean_g_5=data.query('target == "good" and clean == 5')
clean_g=[len(clean_g_1),len(clean_g_2),len(clean_g_3),len(clean_g_4),len(clean_g_5)]
clean_g

[112, 535, 2233, 8942, 28326]

In [131]:
#numero di volte per ogni valore di clean dato bad
clean_b_1=data.query('target == "bad" and clean == 1')
clean_b_2=data.query('target == "bad" and clean == 2')
clean_b_3=data.query('target == "bad" and clean == 3')
clean_b_4=data.query('target == "bad" and clean == 4')
clean_b_5=data.query('target == "bad" and clean == 5')
clean_b=[len(clean_b_1),len(clean_b_2),len(clean_b_3),len(clean_b_4),len(clean_b_5)]
clean_b

[1689, 2061, 3807, 6295, 10846]

In [132]:
#numero di volte per ogni valore di value dato good
value_g_1=data.query('target == "good" and value == 1')
value_g_2=data.query('target == "good" and value == 2')
value_g_3=data.query('target == "good" and value == 3')
value_g_4=data.query('target == "good" and value == 4')
value_g_5=data.query('target == "good" and value == 5')
value_g=[len(value_g_1),len(value_g_2),len(value_g_3),len(value_g_4),len(value_g_5)]

In [133]:
#numero di volte per ogni valore di value dato bad
value_b_1=data.query('target == "bad" and value == 1')
value_b_2=data.query('target == "bad" and value == 2')
value_b_3=data.query('target == "bad" and value == 3')
value_b_4=data.query('target == "bad" and value == 4')
value_b_5=data.query('target == "bad" and value == 5')
value_b=[len(value_b_1),len(value_b_2),len(value_b_3),len(value_b_4),len(value_b_5)]

In [134]:
#numero di volte per ogni valore di room dato good
room_g_1=data.query('target == "good" and room == 1')
room_g_2=data.query('target == "good" and room == 2')
room_g_3=data.query('target == "good" and room == 3')
room_g_4=data.query('target == "good" and room == 4')
room_g_5=data.query('target == "good" and room == 5')
room_g=[len(room_g_1),len(room_g_2),len(room_g_3),len(room_g_4),len(room_g_5)]

In [135]:
#numero di volte per ogni valore di room dato bad
room_b_1=data.query('target == "bad" and room == 1')
room_b_2=data.query('target == "bad" and room == 2')
room_b_3=data.query('target == "bad" and room == 3')
room_b_4=data.query('target == "bad" and room == 4')
room_b_5=data.query('target == "bad" and room == 5')
room_b=[len(room_b_1),len(room_b_2),len(room_b_3),len(room_b_4),len(room_b_5)]

In [136]:
#numero di volte per ogni valore di location dato good
loc_g_1=data.query('target == "good" and location == 1')
loc_g_2=data.query('target == "good" and location == 2')
loc_g_3=data.query('target == "good" and location == 3')
loc_g_4=data.query('target == "good" and location == 4')
loc_g_5=data.query('target == "good" and location == 5')
loc_g=[len(loc_g_1),len(loc_g_2),len(loc_g_3),len(loc_g_4),len(loc_g_5)]

In [137]:
#numero di volte per ogni valore di location dato bad
loc_b_1=data.query('target == "bad" and location == 1')
loc_b_2=data.query('target == "bad" and location == 2')
loc_b_3=data.query('target == "bad" and location == 3')
loc_b_4=data.query('target == "bad" and location == 4')
loc_b_5=data.query('target == "bad" and location == 5')
loc_b=[len(loc_b_1),len(loc_b_2),len(loc_b_3),len(loc_b_4),len(loc_b_5)]

In [138]:
#numero di volte per ogni valore di check dato good
check_g_1=data.query('target == "good" and check == 1')
check_g_2=data.query('target == "good" and check == 2')
check_g_3=data.query('target == "good" and check == 3')
check_g_4=data.query('target == "good" and check == 4')
check_g_5=data.query('target == "good" and check == 5')
check_g=[len(check_g_1),len(check_g_2),len(check_g_3),len(check_g_4),len(check_g_5)]

In [139]:
#numero di volte per ogni valore di check dato bad
check_b_1=data.query('target == "bad" and check == 1')
check_b_2=data.query('target == "bad" and check == 2')
check_b_3=data.query('target == "bad" and check == 3')
check_b_4=data.query('target == "bad" and check == 4')
check_b_5=data.query('target == "bad" and check == 5')
check_b=[len(check_b_1),len(check_b_2),len(check_b_3),len(check_b_4),len(check_b_5)]

In [140]:
#numero di volte per ogni valore di service dato good
ser_g_1=data.query('target == "good" and service == 1')
ser_g_2=data.query('target == "good" and service == 2')
ser_g_3=data.query('target == "good" and service == 3')
ser_g_4=data.query('target == "good" and service == 4')
ser_g_5=data.query('target == "good" and service == 5')
ser_g=[len(ser_g_1),len(ser_g_2),len(ser_g_3),len(ser_g_4),len(ser_g_5)]

In [141]:
#numero di volte per ogni valore di service dato bad
ser_b_1=data.query('target == "bad" and service == 1')
ser_b_2=data.query('target == "bad" and service == 2')
ser_b_3=data.query('target == "bad" and service == 3')
ser_b_4=data.query('target == "bad" and service == 4')
ser_b_5=data.query('target == "bad" and service == 5')
ser_b=[len(ser_b_1),len(ser_b_2),len(ser_b_3),len(ser_b_4),len(ser_b_5)]

In [142]:
#numero di volte per ogni valore di business dato good
b_g_1=data.query('target == "good" and business == 1')
b_g_2=data.query('target == "good" and business == 2')
b_g_3=data.query('target == "good" and business == 3')
b_g_4=data.query('target == "good" and business == 4')
b_g_5=data.query('target == "good" and business == 5')
b_g=[len(b_g_1),len(b_g_2),len(b_g_3),len(b_g_4),len(b_g_5)]

In [143]:
#numero di volte per ogni valore di service dato bad
b_b_1=data.query('target == "bad" and business == 1')
b_b_2=data.query('target == "bad" and business == 2')
b_b_3=data.query('target == "bad" and business == 3')
b_b_4=data.query('target == "bad" and business == 4')
b_b_5=data.query('target == "bad" and business == 5')
b_b=[len(b_b_1),len(b_b_2),len(b_b_3),len(b_b_4),len(b_b_5)]

In [144]:
#create view to cpt
def generate_cpt(name,array_g,array_b):
    cpt= PrettyTable()
    cpt.field_names =["P("+name+"|target)",1,2,3,4,5]
    good_c=['good']
    bad_c=['bad']
    for i in range(len(array_g)):
        good_c.append(float(array_g[i])/good_n)
        bad_c.append(float(array_b[i])/bad_n)
    cpt.add_row(good_c)
    cpt.add_row(bad_c)
    return cpt


In [145]:
print(generate_cpt("clean",clean_g,clean_b))

+-----------------+------------------+-----------------+----------------+----------------+----------------+
| P(clean|target) |        1         |        2        |       3        |       4        |       5        |
+-----------------+------------------+-----------------+----------------+----------------+----------------+
|       good      | 0.00278967819069 | 0.0133256949288 | 0.055619208927 | 0.222725914118 | 0.705539503836 |
|       bad       |  0.068386104138  | 0.0834480524739 | 0.154142035792 | 0.254878937566 | 0.43914487003  |
+-----------------+------------------+-----------------+----------------+----------------+----------------+


In [146]:
print(generate_cpt("value",value_g,value_b))

+-----------------+-----------------+-----------------+-----------------+----------------+----------------+
| P(value|target) |        1        |        2        |        3        |       4        |       5        |
+-----------------+-----------------+-----------------+-----------------+----------------+----------------+
|       good      | 0.0057786191093 | 0.0247085782604 | 0.0937282056391 | 0.324250273986 | 0.551534323005 |
|       bad       |  0.116568143169 |  0.137824925095 |  0.169001538586 | 0.268766701757 | 0.307838691392 |
+-----------------+-----------------+-----------------+-----------------+----------------+----------------+


In [147]:
print(generate_cpt("room",room_g,room_b))

+----------------+------------------+-----------------+-----------------+----------------+----------------+
| P(room|target) |        1         |        2        |        3        |       4        |       5        |
+----------------+------------------+-----------------+-----------------+----------------+----------------+
|      good      | 0.00630168377005 | 0.0285194779317 | 0.0958204642822 | 0.322880342732 | 0.546478031284 |
|      bad       |  0.107377115556  |  0.119888250061 |  0.166491213863 | 0.274840068022 | 0.331403352498 |
+----------------+------------------+-----------------+-----------------+----------------+----------------+


In [148]:
print(generate_cpt("location",loc_g,loc_b))

+--------------------+-----------------+-----------------+-----------------+----------------+----------------+
| P(location|target) |        1        |        2        |        3        |       4        |       5        |
+--------------------+-----------------+-----------------+-----------------+----------------+----------------+
|        good        | 0.0034123742154 | 0.0175102122148 | 0.0720832918203 | 0.252092258643 | 0.654901863107 |
|        bad         | 0.0387480767673 | 0.0712203417281 |  0.152684427889 | 0.298323750911 | 0.439023402705 |
+--------------------+-----------------+-----------------+-----------------+----------------+----------------+


In [149]:
print(generate_cpt("check",check_g,check_b))

+-----------------+------------------+-----------------+-----------------+----------------+----------------+
| P(check|target) |        1         |        2        |        3        |       4        |       5        |
+-----------------+------------------+-----------------+-----------------+----------------+----------------+
|       good      | 0.00993822855435 | 0.0258543389459 | 0.0934791272293 | 0.228355086181 | 0.642373219089 |
|       bad       | 0.0986719572435  | 0.0971333711232 |  0.191675439307 | 0.239007207061 | 0.373512025265 |
+-----------------+------------------+-----------------+-----------------+----------------+----------------+


In [150]:
print(generate_cpt("service",ser_g,ser_b))

+-------------------+------------------+-----------------+-----------------+----------------+----------------+
| P(service|target) |        1         |        2        |        3        |       4        |       5        |
+-------------------+------------------+-----------------+-----------------+----------------+----------------+
|        good       | 0.00757198366046 | 0.0229401215503 | 0.0794560127528 | 0.241107900767 | 0.648923981269 |
|        bad        |  0.109563527411  |  0.10081787999  |  0.173779253381 | 0.242853672362 | 0.372985666856 |
+-------------------+------------------+-----------------+-----------------+----------------+----------------+


In [151]:
print(generate_cpt("business",b_g,b_b))

+--------------------+-----------------+----------------+----------------+----------------+----------------+
| P(business|target) |        1        |       2        |       3        |       4        |       5        |
+--------------------+-----------------+----------------+----------------+----------------+----------------+
|        good        | 0.0163395436884 | 0.042692039454 | 0.212015542493 | 0.27998904055  | 0.448963833815 |
|        bad         |  0.10701271358  | 0.107336626447 | 0.269657462143 | 0.250465624747 | 0.265527573083 |
+--------------------+-----------------+----------------+----------------+----------------+----------------+


In [174]:
#test set con target basato sull'assunzione overall<4 = false 
data=pd.read_csv("./csv/all_review_test.csv")
sentimental=pd.read_csv("./csv/views_test.csv")
data_sentimental=pd.read_csv("./csv/all_review_test.csv")

del data_sentimental['target']
data_sentimental['target']=sentimental['sentimental']

data.query('overall > 0',inplace=True)
data.query('clean > 0',inplace=True)
data.query('value >0',inplace=True)
data.query('room >0',inplace=True)
data.query('location >0',inplace=True)
data.query('check >0',inplace=True)
data.query('service >0',inplace=True)
data.query('business >0',inplace=True)
#data['id'] = np.arange(len(data))
del data['id']
data['id'] = np.arange(len(data))

cols=data.columns.tolist()
cols= cols[-1:]+cols[:-1]
data=data[cols]
data.to_csv("./csv/test_target.csv",index=False)
all_data=len(data)
data.head(15)

,id,hotel,reader,helpful,overall,value,room,location,clean,check,service,business,target
2,0,hotel_80799,-1,-1,3,5,3,3,3,5,3,4,bad
5,1,hotel_80799,1,1,1,1,1,1,1,3,2,1,bad
6,2,hotel_80799,2,1,4,5,3,4,5,5,5,3,good
9,3,hotel_80799,-1,-1,1,1,1,1,3,2,3,2,bad
13,4,hotel_80799,3,3,1,2,1,3,2,5,5,1,bad
16,5,hotel_80799,1,1,1,1,1,1,1,2,2,2,bad
17,6,hotel_80799,1,1,1,1,1,1,1,2,2,3,bad
22,7,hotel_80799,8,4,5,5,5,5,5,5,5,5,good
23,8,hotel_80799,9,2,4,4,4,2,4,4,4,3,good
28,9,hotel_80799,4,0,4,5,3,4,5,4,5,3,good


In [175]:
data_sentimental.query('overall > 0',inplace=True)
data_sentimental.query('clean > 0',inplace=True)
data_sentimental.query('value >0',inplace=True)
data_sentimental.query('room >0',inplace=True)
data_sentimental.query('location >0',inplace=True)
data_sentimental.query('check >0',inplace=True)
data_sentimental.query('service >0',inplace=True)
data_sentimental.query('business >0',inplace=True)
#data['id'] = np.arange(len(data))
del data_sentimental['id']
data_sentimental['id'] = np.arange(len(data_sentimental))

cols=data_sentimental.columns.tolist()
cols= cols[-1:]+cols[:-1]
data_sentimental=data_sentimental[cols]
data_sentimental.to_csv("./csv/test_target_sentimental.csv",index=False)
all_data_sent=len(data_sentimental)
data_sentimental.head(15)

,id,hotel,reader,helpful,overall,value,room,location,clean,check,service,business,target
2,0,hotel_80799,-1,-1,3,5,3,3,3,5,3,4,bad
5,1,hotel_80799,1,1,1,1,1,1,1,3,2,1,bad
6,2,hotel_80799,2,1,4,5,3,4,5,5,5,3,good
9,3,hotel_80799,-1,-1,1,1,1,1,3,2,3,2,bad
13,4,hotel_80799,3,3,1,2,1,3,2,5,5,1,bad
16,5,hotel_80799,1,1,1,1,1,1,1,2,2,2,bad
17,6,hotel_80799,1,1,1,1,1,1,1,2,2,3,bad
22,7,hotel_80799,8,4,5,5,5,5,5,5,5,5,bad
23,8,hotel_80799,9,2,4,4,4,2,4,4,4,3,good
28,9,hotel_80799,4,0,4,5,3,4,5,4,5,3,good


In [176]:
data=pd.read_csv("./csv/all_review_test.csv")

data.query('overall > 0',inplace=True)
data.query('clean > 0',inplace=True)
data.query('value >0',inplace=True)
data.query('room >0',inplace=True)
data.query('location >0',inplace=True)
data.query('check >0',inplace=True)
data.query('service >0',inplace=True)
data.query('business >0',inplace=True)
#data['id'] = np.arange(len(data))
del data['id']
data['id'] = np.arange(len(data))
del data['target']
cols=data.columns.tolist()
cols= cols[-1:]+cols[:-1]
data=data[cols]
data.to_csv("./csv/test_not_target.csv",index=False)
all_data=len(data)
data.head()

,id,hotel,reader,helpful,overall,value,room,location,clean,check,service,business
2,0,hotel_80799,-1,-1,3,5,3,3,3,5,3,4
5,1,hotel_80799,1,1,1,1,1,1,1,3,2,1
6,2,hotel_80799,2,1,4,5,3,4,5,5,5,3
9,3,hotel_80799,-1,-1,1,1,1,1,3,2,3,2
13,4,hotel_80799,3,3,1,2,1,3,2,5,5,1


In [177]:
skel= GraphSkeleton()
nd =NodeData()

In [178]:
skel.load('./skeleton.txt')
nd.load("./skeleton.txt")

In [179]:
skel.toporder()
bn=DiscreteBayesianNetwork(skel, nd)

In [180]:
skel.getparents("room")

[u'target']

In [181]:
skel.getchildren("target")

[u'clean', u'value', u'room', u'location', u'check', u'service', u'business']

In [182]:
row=data.query('id==0')
result_row=row.values.tolist()[0]
result_row

[0, ' hotel_80799', -1, -1, 3, 5, 3, 3, 3, 5, 3, 4]

In [183]:
def inference(row):
    fn =TableCPDFactorization(bn)
    evidence =dict(clean=row[8],value=row[5],room=row[6],location=row[7],check=row[9],service=row[10],business=row[11])
    query= {'target' :['bad']}
    bad = fn.specificquery(query,evidence)
    good=1-bad
    return (good,bad)

In [184]:
list_result=[]
list_good=[]
for i in range(14161):
    row=data.query('id=='+str(i))
    result_row=row.values.tolist()[0]
    good,bad=inference(result_row)
    print(good,bad)
    list_good.append(good)
    if good < bad:
        list_result.append('bad')
    else:
        list_result.append('good')
data['target']=list_result
        
    

(0.6643632899133014, 0.33563671008669854)
(1.0380585280245214e-13, 0.9999999999998962)
(0.9998042580038344, 0.00019574199616557777)
(4.5175196916602545e-11, 0.9999999999548248)
(1.6074890475392678e-06, 0.9999983925109525)
(1.5298873279334657e-13, 0.999999999999847)
(4.812816811750054e-13, 0.9999999999995187)
(0.9999999201142595, 7.988574043534107e-08)
(0.6349463189210713, 0.3650536810789286)
(0.9988220595785147, 0.0011779404214853012)
(0.8342694756514878, 0.16573052434851224)
(0.002322800296220895, 0.9976771997037791)
(3.427554341239869e-06, 0.9999965724456588)
(9.909228490312394e-06, 0.9999900907715097)
(0.00012216272071363665, 0.9998778372792864)
(7.034373084024992e-13, 0.9999999999992966)
(3.5860203695392556e-14, 0.9999999999999641)
(8.881784197001252e-16, 0.9999999999999991)
(0.24788565272715912, 0.7521143472728409)
(8.881784197001252e-16, 0.9999999999999991)
(2.0718053153290228e-05, 0.9999792819468467)
(8.019007680104551e-11, 0.9999999999198099)
(9.325873406851315e-14, 0.999999999

(0.9999576203801896, 4.2379619810337395e-05)
(0.9809706554276924, 0.019029344572307674)
(0.9999999201142595, 7.988574043534107e-08)
(0.01362920748779084, 0.9863707925122092)
(0.46839661036910085, 0.5316033896308991)
(0.7005104090272471, 0.29948959097275285)
(0.9997528699846521, 0.00024713001534789757)
(0.9973588522589406, 0.002641147741059423)
(0.1669193787388198, 0.8330806212611802)
(8.881784197001252e-16, 0.9999999999999991)
(2.622532103702113e-07, 0.9999997377467896)
(0.0024704351377533573, 0.9975295648622466)
(0.9192530506010486, 0.08074694939895147)
(0.999999689081971, 3.1091802894448724e-07)
(0.9998902401932361, 0.000109759806763859)
(0.9991732661696129, 0.0008267338303870758)
(0.9680744249400723, 0.03192557505992768)
(0.012233932966548955, 0.987766067033451)
(0.9970147412309754, 0.0029852587690245135)
(7.597944495785214e-11, 0.9999999999240206)
(0.9999773546884215, 2.2645311578520543e-05)
(1.058342699145065e-06, 0.9999989416573009)
(0.8581290513439903, 0.14187094865600966)
(0.99

(0.9999996782520912, 3.217479087528409e-07)
(0.8048457262926365, 0.19515427370736346)
(0.9999959921891364, 4.0078108635740614e-06)
(0.0004941255489234742, 0.9995058744510765)
(0.9981054142622419, 0.0018945857377581222)
(0.9999837128711153, 1.6287128884651634e-05)
(0.3562356820825716, 0.6437643179174284)
(0.9999952449471524, 4.755052847554658e-06)
(0.9999504016464259, 4.959835357405898e-05)
(0.07079946643544421, 0.9292005335645558)
(0.9088783538799441, 0.09112164612005597)
(3.050320773745341e-09, 0.9999999969496792)
(0.9415445468886131, 0.05845545311138693)
(0.9999989702500721, 1.0297499278497248e-06)
(5.486123685782829e-06, 0.9999945138763142)
(0.9995512009371478, 0.00044879906285225514)
(0.999991346375934, 8.65362406601846e-06)
(3.938817794146843e-08, 0.9999999606118221)
(0.9993932795471954, 0.0006067204528046789)
(0.9999999201142595, 7.988574043534107e-08)
(0.9999989702500721, 1.0297499278497248e-06)
(0.9999674503354686, 3.25496645313977e-05)
(0.9999937970869678, 6.202913032253166e-0

(0.9933888128523779, 0.00661118714762213)
(0.8824814249459458, 0.11751857505405416)
(0.9999981271137719, 1.8728862281313847e-06)
(0.9998965359313292, 0.0001034640686707686)
(0.4694205787664222, 0.5305794212335778)
(8.028396673087634e-05, 0.9999197160332691)
(0.9999980618776064, 1.9381223935255e-06)
(0.9999999201142595, 7.988574043534107e-08)
(1.1472586922689487e-06, 0.9999988527413077)
(4.7454815332859113e-07, 0.9999995254518467)
(0.7374589227056969, 0.26254107729430315)
(0.999999518789318, 4.812106819457903e-07)
(0.9992010714090725, 0.0007989285909274029)
(0.993895333519708, 0.006104666480291988)
(0.014988806407641886, 0.9850111935923581)
(0.9994072603116639, 0.0005927396883360081)
(0.9999961520961447, 3.84790385532156e-06)
(0.9999911303331038, 8.869666896194924e-06)
(0.9999999201142595, 7.988574043534107e-08)
(0.9999999201142595, 7.988574043534107e-08)
(0.9993614279223627, 0.0006385720776372731)
(0.001176264210222988, 0.998823735789777)
(0.7374589227056969, 0.26254107729430315)
(0.00

(0.999154605145754, 0.0008453948542459614)
(3.1735678907018894e-07, 0.9999996826432109)
(0.9999940977626174, 5.9022373826382074e-06)
(0.9999804936243786, 1.9506375621433947e-05)
(1.3500089746099064e-07, 0.9999998649991025)
(0.9807065288910305, 0.019293471108969517)
(0.9915762831484722, 0.008423716851527732)
(0.9999996782520912, 3.217479087528409e-07)
(0.9999862723996342, 1.3727600365810193e-05)
(0.5073691589305724, 0.49263084106942756)
(0.9013014817623072, 0.09869851823769285)
(0.9999804641392743, 1.9535860725643193e-05)
(0.9994072603116639, 0.0005927396883360081)
(0.9998271393050591, 0.0001728606949409468)
(0.12141331349270135, 0.8785866865072987)
(1.7021665236205408e-05, 0.9999829783347638)
(0.9999940977626174, 5.9022373826382074e-06)
(0.953665486339535, 0.04633451366046501)
(0.9840800631953341, 0.015919936804665854)
(0.9999644475148314, 3.555248516858393e-05)
(4.192646230194441e-12, 0.9999999999958074)
(0.1304551750694265, 0.8695448249305735)
(0.999991346375934, 8.65362406601846e-06

...

(0.9423263230991828, 0.05767367690081711)
(0.9999999201142595, 7.988574043534107e-08)
(0.999999518789318, 4.812106819457903e-07)
(0.9999999201142595, 7.988574043534107e-08)
(0.9999972961396104, 2.703860389561209e-06)
(0.013856008838597389, 0.9861439911614026)
(0.9999989702500721, 1.0297499278497248e-06)
(0.999999689081971, 3.1091802894448724e-07)
(0.9999999201142595, 7.988574043534107e-08)
(0.9999999201142595, 7.988574043534107e-08)
(0.9999999201142595, 7.988574043534107e-08)
(0.6770823064820802, 0.32291769351791977)
(0.9999996782520912, 3.217479087528409e-07)
(0.2822792769288227, 0.7177207230711773)
(0.9999999201142595, 7.988574043534107e-08)
(0.9999888912745064, 1.1108725493615802e-05)
(0.09458586731813834, 0.9054141326818617)
(0.9999723601350949, 2.763986490508432e-05)
(0.9842987456411909, 0.015701254358809064)
(0.1301537389883377, 0.8698462610116623)
(0.9998831965620869, 0.00011680343791306508)
(0.9999873774783128, 1.2622521687201222e-05)
(0.9999981558251714, 1.8441748286457409e-06

(1.0093234167118936e-05, 0.9999899067658329)
(0.19642923309975213, 0.8035707669002479)
(0.999938709422703, 6.129057729698468e-05)
(0.9999999201142595, 7.988574043534107e-08)
(0.9999876795909087, 1.2320409091291029e-05)
(0.633900770928153, 0.36609922907184705)
(0.9013014817623072, 0.09869851823769285)
(0.9965420788829694, 0.0034579211170306184)
(1.0595706325666754e-06, 0.9999989404293674)
(0.03169720841311674, 0.9683027915868833)
(0.9999961520961447, 3.84790385532156e-06)
(0.9999999201142595, 7.988574043534107e-08)
(1.4027183858900116e-09, 0.9999999985972816)
(0.999999518789318, 4.812106819457903e-07)
(0.9999967568020438, 3.243197956227045e-06)
(0.995481197454345, 0.004518802545654962)
(0.9993326018525703, 0.0006673981474296302)
(0.9999972418283595, 2.7581716405152647e-06)
(0.9981124703846832, 0.0018875296153168354)
(0.9999845023543859, 1.5497645614141843e-05)
(0.9999999201142595, 7.988574043534107e-08)
(0.999991346375934, 8.65362406601846e-06)
(0.00010275345006094394, 0.999897246549939

(0.9999989702500721, 1.0297499278497248e-06)
(0.9919733043759521, 0.008026695624047883)
(0.9999961520961447, 3.84790385532156e-06)
(0.9917410241581007, 0.008258975841899309)
(0.7672383239287577, 0.23276167607124237)
(0.6142745253559452, 0.38572547464405477)
(6.672440377997191e-14, 0.9999999999999333)
(0.999910736192637, 8.926380736293184e-05)
(0.996404296128974, 0.0035957038710260772)
(2.3135716942235263e-07, 0.9999997686428306)
(0.967994311349389, 0.03200568865061103)
(7.628397913350682e-11, 0.999999999923716)
(0.9999959560732784, 4.04392672160919e-06)
(0.9958612956280191, 0.004138704371980979)
(0.9991311864822963, 0.0008688135177037703)
(0.9999967568020438, 3.243197956227045e-06)
(0.9991732661696129, 0.0008267338303870758)
(0.9999999201142595, 7.988574043534107e-08)
(0.013689489497286766, 0.9863105105027132)
(0.9999989702500721, 1.0297499278497248e-06)
(5.50194154936845e-05, 0.9999449805845063)
(0.9999996782520912, 3.217479087528409e-07)
(0.9998965359313292, 0.0001034640686707686)
(0

(0.9998846493474375, 0.00011535065256249009)
(0.9998846493474375, 0.00011535065256249009)
(0.9415445468886131, 0.05845545311138693)
(0.03307856947380372, 0.9669214305261963)
(0.8992873388410725, 0.10071266115892756)
(0.9999987477470311, 1.2522529689019236e-06)
(0.9811710810497429, 0.01882891895025711)
(0.014054083747018553, 0.9859459162529814)
(0.19642923309975213, 0.8035707669002479)
(0.9998846493474375, 0.00011535065256249009)
(0.9977727059691883, 0.0022272940308117107)
(0.9999366129094382, 6.33870905616991e-05)
(0.9999996782520912, 3.217479087528409e-07)
(0.9999999201142595, 7.988574043534107e-08)
(5.685307780112225e-10, 0.9999999994314692)
(0.9999981558251714, 1.8441748286457409e-06)
(0.9999303914169553, 6.960858304477476e-05)
(0.9999901718854781, 9.828114521912061e-06)
(2.3376856006507296e-12, 0.9999999999976623)
(0.9999937970869678, 6.202913032253166e-06)
(0.999991346375934, 8.65362406601846e-06)
(0.7212467009364467, 0.27875329906355323)
(0.9778261162616619, 0.022173883738338052)

(0.9998358505979199, 0.00016414940208006577)
(0.01100341039052588, 0.9889965896094741)
(2.1027624086400465e-13, 0.9999999999997897)
(0.9999996782520912, 3.217479087528409e-07)
(6.854545375745147e-09, 0.9999999931454546)
(5.944376313404831e-08, 0.9999999405562369)
(6.580978095804824e-09, 0.9999999934190219)
(0.5493684009313053, 0.4506315990686946)
(3.862849546232283e-06, 0.9999961371504538)
(0.0002817460459997312, 0.9997182539540003)
(0.9999999201142595, 7.988574043534107e-08)
(0.9999972961396104, 2.703860389561209e-06)
(0.9991732661696129, 0.0008267338303870758)
(0.9999967616970206, 3.2383029794080004e-06)
(0.5619331476840213, 0.43806685231597875)
(9.684801712173119e-05, 0.9999031519828783)
(0.9991732661696129, 0.0008267338303870758)
(0.7374589227056969, 0.26254107729430315)
(0.9643212844161597, 0.0356787155838403)
(0.9999995421153136, 4.5788468633979947e-07)
(0.3486515961046659, 0.6513484038953341)
(0.9999940977626174, 5.9022373826382074e-06)
(0.9940659339576177, 0.0059340660423823485

In [187]:
data.head()
data.to_csv("./csv/final_test.csv",index=False)

In [188]:
len(list_good)

14161

In [189]:
f= open('./probs/probs.csv','w')
for i in list_good:
    f.write(str(i)+'\n')

In [166]:
#inferenza con naive bayes con sentimental analysis
skel= GraphSkeleton()
nd =NodeData()

In [167]:
skel.load('./skeleton_sentimental.txt')
nd.load("./skeleton_sentimental.txt")

In [168]:
skel.toporder()
bn=DiscreteBayesianNetwork(skel, nd)

In [169]:
list_result=[]
list_good_sent=[]
for i in range(len(data)):
    row=data.query('id=='+str(i))
    result_row=row.values.tolist()[0]
    good,bad=inference(result_row)
    list_good_sent.append(good)
    print(good,bad)
    if good < bad:
        list_result.append('bad')
    else:
        list_result.append('good')
data['target']=list_result
        

(0.20063685280457966, 0.7993631471954203)
(2.878688639684768e-07, 0.999999712131136)
(0.8426081442301987, 0.15739185576980128)
(7.274296986325979e-06, 0.9999927257030137)
(0.000588726831329045, 0.999411273168671)
(4.092661721744051e-07, 0.9999995907338278)
(8.090250555037315e-07, 0.9999991909749445)
(0.9831170008158527, 0.01688299918414722)
(0.2696250872070326, 0.7303749127929674)
(0.7483727310780395, 0.25162726892196047)
(0.3402490015419962, 0.6597509984580038)
(0.032610045490661155, 0.9673899545093388)
(0.0026018423942621993, 0.9973981576057378)
(0.0012196514757664145, 0.9987803485242336)
(0.006362325931454915, 0.9936376740685451)
(7.463610506297513e-07, 0.9999992536389494)
(1.5711279166552572e-07, 0.9999998428872083)
(1.8057073947197466e-08, 0.999999981942926)
(0.28889985987592615, 0.7111001401240739)
(1.8057073947197466e-08, 0.999999981942926)
(0.003959115483989639, 0.9960408845160104)
(5.664482972123075e-06, 0.9999943355170279)
(2.3272829652309923e-07, 0.9999997672717035)
(0.00055

(0.8514645460873611, 0.14853545391263895)
(0.48141516207638213, 0.5185848379236179)
(0.06373130840528529, 0.9362686915947147)
(0.7840852247794532, 0.21591477522054675)
(9.01552441878728e-06, 0.9999909844755812)
(0.8966993576468401, 0.10330064235315996)
(0.0013824699532064555, 0.9986175300467935)
(0.4691510973375489, 0.5308489026624511)
(0.9043118915905038, 0.09568810840949618)
(0.5003398740853324, 0.4996601259146675)
(0.0010462898209201432, 0.9989537101790799)
(0.974682751597515, 0.025317248402484963)
(2.6346458536830042e-05, 0.9999736535414632)
(0.24086414668183864, 0.7591358533181614)
(0.004404376154894418, 0.9955956238451056)
(0.9364175672546415, 0.06358243274535852)
(0.9480067745974612, 0.0519932254025388)
(0.6999510184756521, 0.3000489815243479)
(0.6991519681090824, 0.3008480318909176)
(0.0007808067824580656, 0.9992191932175419)
(0.8941890928157619, 0.10581090718423819)
(0.7683444049702215, 0.23165559502977853)
(0.8012638932207498, 0.19873610677925027)
(0.8254589000846926, 0.17454

(0.7882487304470237, 0.21175126955297627)
(0.9831170008158527, 0.01688299918414722)
(0.9643843383202898, 0.03561566167971015)
(0.884772264979308, 0.11522773502069197)
(0.9376663786659557, 0.06233362133404425)
(0.685337655263812, 0.314662344736188)
(0.33921095560841474, 0.6607890443915853)
(0.18131161587295996, 0.81868838412704)
(0.0006343871392809897, 0.999365612860719)
(0.4646025538932125, 0.5353974461067875)
(0.48096991726312677, 0.5190300827368732)
(9.307058797980439e-06, 0.999990692941202)
(0.7161240785404701, 0.2838759214595299)
(0.9751313638215968, 0.024868636178403212)
(0.11172354416133234, 0.8882764558386677)
(0.3449211318037587, 0.6550788681962413)
(0.6475231030264641, 0.3524768969735359)
(0.9255433220269285, 0.07445667797307148)
(0.05597090965129836, 0.9440290903487016)
(0.21003453237616343, 0.7899654676238366)
(0.8915206354179812, 0.10847936458201875)
(0.005918030413812447, 0.9940819695861876)
(0.27189410614489673, 0.7281058938551033)
(0.9428989366806199, 0.05710106331938017

(0.0569033514809425, 0.9430966485190575)
(0.9831170008158527, 0.01688299918414722)
(1.532601932141553e-06, 0.9999984673980679)
(0.9382498097008264, 0.061750190299173606)
(0.9707852302595866, 0.029214769740413322)
(0.1262225098585138, 0.8737774901414862)
(0.8915206354179812, 0.10847936458201875)
(7.73872141123233e-05, 0.9999226127858877)
(0.9399455642053496, 0.06005443579465037)
(0.8525128125299857, 0.1474871874700143)
(0.006364287306846128, 0.9936357126931539)
(0.974682751597515, 0.025317248402484963)
(0.5592415441040524, 0.4407584558959477)
(0.4793948527368823, 0.5206051472631177)
(0.9831170008158527, 0.01688299918414722)
(0.9705767164024883, 0.029423283597511642)
(0.03322849977779041, 0.9667715002222096)
(0.818857281297507, 0.18114271870249293)
(0.010009603032864356, 0.9899903969671356)
(0.9831170008158527, 0.01688299918414722)
(9.680119217314598e-08, 0.9999999031988078)
(0.7492900972964647, 0.25070990270353527)
(0.8965760238271759, 0.10342397617282409)
(0.8012638932207498, 0.1987361

(0.9831170008158527, 0.01688299918414722)
(0.22255442594510844, 0.7774455740548916)
(0.9831170008158527, 0.01688299918414722)
(9.307058797980439e-06, 0.999990692941202)
(0.7457571328896337, 0.25424286711036626)
(0.8815677172060103, 0.11843228279398967)
(0.4287216030043013, 0.5712783969956987)
(0.12749231070958733, 0.8725076892904127)
(0.7406383373236805, 0.2593616626763196)
(0.13657401737809882, 0.8634259826219012)
(0.00011653756088392253, 0.9998834624391161)
(0.974682751597515, 0.025317248402484963)
(0.030175706973593064, 0.9698242930264069)
(0.051538031978495935, 0.9484619680215041)
(0.9831170008158527, 0.01688299918414722)
(0.3202786640595625, 0.6797213359404375)
(0.9133398847579345, 0.08666011524206549)
(0.5592415441040524, 0.4407584558959477)
(0.9240713007566053, 0.07592869924339471)
(0.003064291416411402, 0.9969357085835886)
(0.9553325712942342, 0.04466742870576574)
(0.03789206642424092, 0.9621079335757591)
(0.0621574524640085, 0.9378425475359915)
(1.0014227864729719e-06, 0.99999

...

(0.2550849508304204, 0.7449150491695796)
(0.9831170008158527, 0.01688299918414722)
(0.9384135873729192, 0.06158641262708079)
(0.10878163758657311, 0.8912183624134269)
(0.9206204244530244, 0.07937957554697565)
(0.6913421205210657, 0.3086578794789343)
(0.14619475496527057, 0.8538052450347294)
(0.909465354806008, 0.09053464519399204)
(0.9306068383421066, 0.06939316165789343)
(0.9648232873290508, 0.03517671267094927)
(0.0033077949531367112, 0.9966922050468633)
(4.571800712338536e-05, 0.9999542819928766)
(0.9831170008158527, 0.01688299918414722)
(0.7629930605997716, 0.23700693940022846)
(0.31087623773208806, 0.6891237622679119)
(0.8760128274780828, 0.12398717252191714)
(0.6913421205210657, 0.3086578794789343)
(0.6710080704666873, 0.3289919295333127)
(0.35275693860787294, 0.6472430613921271)
(0.14427134519606177, 0.8557286548039382)
(0.6242889947222814, 0.3757110052777186)
(0.9129865808360127, 0.08701341916398733)
(0.9764628237938611, 0.023537176206138908)
(0.5231939994822526, 0.476806000517

(0.9831170008158527, 0.01688299918414722)
(0.9480067745974612, 0.0519932254025388)
(0.00865867212474547, 0.9913413278752545)
(0.9040577024056082, 0.09594229759439182)
(0.9387961100178852, 0.061203889982114824)
(0.8679230347224071, 0.13207696527759294)
(0.9831170008158527, 0.01688299918414722)
(0.00020934798512139974, 0.9997906520148786)
(0.6543466553511053, 0.3456533446488948)
(2.1341785034434402e-05, 0.9999786582149656)
(0.9561086212215382, 0.043891378778461845)
(0.12649550375223106, 0.8735044962477689)
(0.9564632098991986, 0.043536790100801394)
(0.9648232873290508, 0.03517671267094927)
(0.7587742361523176, 0.24122576384768232)
(0.8012638932207498, 0.19873610677925027)
(0.9831170008158527, 0.01688299918414722)
(0.2042166854973867, 0.7957833145026133)
(0.9831170008158527, 0.01688299918414722)
(0.9831170008158527, 0.01688299918414722)
(0.5077577784529685, 0.4922422215470315)
(0.7557231960122205, 0.24427680398777954)
(0.0052442061798559925, 0.994755793820144)
(0.008176704905560106, 0.991

(0.0875275084800371, 0.9124724915199629)
(0.8232111219671152, 0.1767888780328848)
(0.06871531992722046, 0.9312846800727795)
(0.8712816925013794, 0.1287183074986206)
(0.9831170008158527, 0.01688299918414722)
(0.6966368194492638, 0.30336318055073624)
(0.757379185280279, 0.242620814719721)
(0.4339742566223548, 0.5660257433776452)
(0.08762888387477707, 0.9123711161252229)
(0.7622330009441612, 0.23776699905583878)
(0.950717081613854, 0.049282918386145916)
(0.9831170008158527, 0.01688299918414722)
(0.6954863857568097, 0.3045136142431904)
(0.13987902452598988, 0.8601209754740101)
(0.9428989366806199, 0.05710106331938017)
(0.9242696482688427, 0.07573035173115727)
(0.9474388018916251, 0.05256119810837492)
(0.8379377914346072, 0.16206220856539275)
(0.9240713007566053, 0.07592869924339471)
(0.9831170008158527, 0.01688299918414722)
(0.009025744569050698, 0.9909742554309493)
(0.8028047917931073, 0.19719520820689274)
(0.00014902276487216426, 0.9998509772351278)
(0.22942344974170492, 0.77057655025829

(0.7168868964848526, 0.2831131035151474)
(0.9831170008158527, 0.01688299918414722)
(0.9167242145005621, 0.08327578549943794)
(0.0040860631415196025, 0.9959139368584804)
(0.9831170008158527, 0.01688299918414722)
(0.020104323106926114, 0.9798956768930739)
(0.7899340309564951, 0.21006596904350489)
(0.9117290153545909, 0.08827098464540904)
(0.891088984161492, 0.10891101583850807)
(0.7683444049702215, 0.23165559502977853)
(0.8978693144823908, 0.10213068551760923)
(0.00046331388815112895, 0.9995366861118489)
(0.9831170008158527, 0.01688299918414722)
(0.15761246849658728, 0.8423875315034127)
(5.0948307123155345e-06, 0.9999949051692877)
(0.4148181819781257, 0.5851818180218743)
(7.707787067112726e-05, 0.9999229221293289)
(0.9831170008158527, 0.01688299918414722)
(0.061842865987090434, 0.9381571340129096)
(9.632481334787002e-06, 0.9999903675186652)
(9.571986183098957e-05, 0.999904280138169)
(4.866074362341877e-07, 0.9999995133925638)
(0.00030245834499331714, 0.9996975416550067)
(0.47865684765533

(0.07683579511253247, 0.9231642048874675)
(0.6433665590887683, 0.3566334409112317)
(0.5229497939853891, 0.47705020601461096)
(0.00103343446621329, 0.9989665655337867)
(0.6133769016012114, 0.38662309839878856)
(0.016818798576587857, 0.9831812014234121)
(0.8514645460873611, 0.14853545391263895)
(0.27123758200417847, 0.7287624179958215)
(0.003446874158950508, 0.9965531258410495)
(0.5334320129613753, 0.4665679870386247)
(0.0009052667132501968, 0.9990947332867498)
(0.6913421205210657, 0.3086578794789343)
(0.974682751597515, 0.025317248402484963)
(0.08083642622281473, 0.9191635737771853)
(0.17426725931394182, 0.8257327406860582)
(0.0024356291315970058, 0.997564370868403)
(0.9643843383202898, 0.03561566167971015)
(0.9480067745974612, 0.0519932254025388)
(0.29779186964988325, 0.7022081303501168)
(0.8095778672735185, 0.1904221327264814)
(0.9117290153545909, 0.08827098464540904)
(0.9129865808360127, 0.08701341916398733)
(0.2297441011936867, 0.7702558988063133)
(0.003770105540932933, 0.9962298944

In [170]:
support=[]
for i in list_good_sent:
    if i > 0:
        support.append(i)

In [171]:
len(support)

14161

In [172]:
import csv
csvfile='./probs/probs_sent.csv'
with open(csvfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in support:
        writer.writerow([val]) 

In [173]:
data.head()
data.to_csv("./csv/final_test_sentimental_naiveb.csv",index=False)